**Data Preparation**

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ratings= pd.read_csv("/content/drive/MyDrive/ML IE506/ml-100k/ml-100k/u.data")
df= pd.DataFrame(ratings)
print(df)
#df['movieId'].max()
#df['movieId'].min()
temp = 1
#userItem  = np.zeros((610+1,df['movieId'].max()+1))
#print(userItem )


        196\t242\t3\t881250949
0       186\t302\t3\t891717742
1        22\t377\t1\t878887116
2        244\t51\t2\t880606923
3       166\t346\t1\t886397596
4       298\t474\t4\t884182806
...                        ...
99994   880\t476\t3\t880175444
99995   716\t204\t5\t879795543
99996  276\t1090\t1\t874795795
99997    13\t225\t2\t882399156
99998    12\t203\t3\t879959583

[99999 rows x 1 columns]


# **Bringing DATA in proper format **

In [29]:
column_names = ['UserID', 'MovieID', 'Rating', 'Timestamp']
data = pd.read_csv('/content/drive/MyDrive/ML IE506/ml-100k/ml-100k/u.data', sep='\t', names=column_names)

# Display the first few rows to verify it's read correctly
data.head()

,UserID,MovieID,Rating,Timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [30]:
# Dropping the 'Timestamp' column
data = data.drop(columns=['Timestamp'])
print(data.head())

   UserID  MovieID  Rating
0     196      242       3
1     186      302       3
2      22      377       1
3     244       51       2
4     166      346       1


In [6]:
a=np.sum(data['Rating'])
print(a)

352986


In [7]:
print(data.shape)
print(data['MovieID'].max())
print(data['MovieID'].min())

(100000, 3)
1682
1


In [8]:
print(data['UserID'].max())

943


In [9]:
userItem  = np.zeros((data['UserID'].max()+1,data['MovieID'].max()+1))
print(userItem )

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [10]:
# Assume data has been loaded and preprocessed as described
import numpy as np

# Initialize the userItem matrix with zeros
userItem = np.zeros((data['UserID'].max()+1, data['MovieID'].max()+1))

# Populate the userItem matrix with all ratings from the dataset
for index, row in data.iterrows():
    userItem[int(row['UserID']), int(row['MovieID'])] = row['Rating']

# Now calculate the sum of the userItem matrix
sum_ratings_userItem = np.sum(userItem)
print(sum_ratings_userItem)


352986.0


**Since 1st row and 1st column doesnot contain any data we will drop them.**

In [11]:
sum1 = np.sum(userItem[:,0])
print(sum1)
sum2 = np.sum(userItem[0,:])
print(sum1)

0.0
0.0


In [12]:
R= (np.delete(userItem,0,1))
R =(np.delete(R,0,0))
print(R)
print(userItem.shape)

[[5. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
(944, 1683)


In [13]:
min_rating = data['Rating'].min()

# Find the maximum value in the "Rating" column
max_rating = data['Rating'].max()

print(f"Minimum Rating: {min_rating}")
print(f"Maximum Rating: {max_rating}")

Minimum Rating: 1
Maximum Rating: 5


# Calculating Item Average

In [14]:
def evaluate_item_averages(R, beta_i, epsilon1, epsilon2, r_min, r_max):
    # Calculate global average
    global_avg = (np.sum(R) + np.random.laplace(5-1/epsilon1)) / 1e5

    # Initialize item averages
    item_averages = np.zeros(R.shape[1])

    for j in range(R.shape[1]):
        # Select ratings for item j
        Rj = R[:, j]

        # Calculate item average
        item_avg = (np.sum(Rj) + beta_i * global_avg + np.random.laplace((r_max - r_min)/epsilon2)) / (Rj.size + beta_i)

        # Clamp item average to specified range
        item_avg = np.clip(item_avg, r_min, r_max)

        item_averages[j] = item_avg

    return item_averages



In [31]:
# Assuming userItem is defined elsewhere and is a NumPy array of shape [num_users, num_items]
# Your function definition here...

# Call the function with the userItem matrix
beta_i = 2  # Example value, adjust as necessary
epsilon1 = 1  # Example value, adjust as necessary
epsilon2 = 1  # Example value, adjust as necessary
r_min = 1  # Assume
r_max = 5  # Assuming 5 is the maximum rating

# Ensure userItem is correctly passed as the R parameter
item_averages = evaluate_item_averages(userItem, beta_i, epsilon1, epsilon2, r_min, r_max)

print("Item averages:", item_averages)
print("Number of items:", len(item_averages))


Item averages: [1.         1.86798189 1.         ... 1.         1.         1.        ]
Number of items: 1683


# Calculating User Average

This code calculates the user averages based on the given ratings matrix R, item averages, stabilization parameter beta_u, and privacy parameters epsilon1 and epsilon2. It also clamps the user averages within the specified range [-2, 2].


In [16]:
def evaluate_user_effects(Re, item_averages, beta_u, epsilon1, epsilon2):
    # Compute R0
    R0 = Re - item_averages
    print(R0.shape)
    # Calculate global average of R0
    global_avg_0 = (np.sum(R0) + np.random.laplace(4/epsilon1)) / 1e5
    print(global_avg_0)
    # Initialize user averages
    user_averages = np.zeros(Re.shape[0])
    print(user_averages.shape)

    for v in range(Re.shape[0]):
        # Select ratings for user v
        Rv = R0[v, :]

        # Calculate user average
        user_avg = (np.sum(Rv) + beta_u * global_avg_0 + np.random.laplace(scale=1/epsilon2)) / (Rv.size + beta_u)

        # Clamp user average to specified range
        user_avg = np.clip(user_avg, -2, 2)

        user_averages[v] = user_avg

    return user_averages





In [17]:
item_averages #Already initialised
beta_u = 2  # Stabilization parameter for user averages
epsilon1 = 1  # Global average privacy parameter
epsilon2 = 1  # User average privacy parameter

user_averages = evaluate_user_effects(userItem, item_averages, beta_u, epsilon1, epsilon2)
print("User averages:", user_averages)
print("Number of items:", len(user_averages))

(944, 1683)
-12.597237424696186
(944,)
User averages: [-1.02829596 -0.44593176 -0.89386834 -0.94019824 -0.96691995 -0.73129262
 -0.57369372 -0.08072233 -0.89592271 -0.97243529 -0.56934044 -0.65775225
 -0.89636417  0.14174861 -0.79099417 -0.85174199 -0.66899096 -0.97843495
 -0.39160725 -0.98662115 -0.94011872 -0.74530514 -0.774839   -0.70357237
 -0.85367469 -0.84142223 -0.84145572 -0.98039005 -0.85419685 -0.95497279
 -0.93314133 -0.94631154 -0.94693519 -0.97496567 -0.98164735 -0.98411705
 -0.98349363 -0.9034745  -0.76248937 -0.97966377 -0.9688349  -0.91261277
 -0.62384286 -0.54058132 -0.70174008 -0.92447405 -0.96537297 -0.97521046
 -0.8817745  -0.68628254 -0.97813527 -0.97865323 -0.88662495 -0.96641178
 -0.88631361 -0.98668181 -0.62416769 -0.80874005 -0.67783129 -0.13662165
 -0.52401573 -0.99128796 -0.57461363 -0.85674506 -0.60296627 -0.84092496
 -0.95013719 -0.96287735 -0.96886641 -0.88031559 -0.76004875 -0.94411195
 -0.71910103 -0.88296081 -0.94456785 -0.87540959 -0.85551907 -0.874779

**Matrix Factorization Algorithm**

In [18]:
R=userItem

# **Differentailly Private SGD**

In [19]:
import numpy as np

def differentially_private_sgd(R, d, gamma, lambd, k, emax, epsilon):
    num_users, num_items = R.shape
    P = np.random.rand(num_users, d)
    Q = np.random.rand(num_items, d)

    delta_r = np.max(R) - np.min(R)

    # Assuming R is a numpy array, find non-zero (i.e., rated) entries
    rated_indices = np.argwhere(R > 0)

    for epoch in range(k):
        np.random.shuffle(rated_indices)  # Shuffle the observed ratings
        for i, j in rated_indices:
            e_ui = R[i, j] - np.dot(P[i], Q[j]) + np.random.laplace(scale=k * delta_r / epsilon)
            e_ui = np.clip(e_ui, -emax, emax)

            # Update rule remains the same, but only applied to observed ratings
            P[i] += gamma * (e_ui * Q[j] - lambd * P[i])
            Q[j] += gamma * (e_ui * P[i] - lambd * Q[j])

    return P, Q


In [20]:
d = 3
gamma = 0.01
lambd = 0.10
k = 5
emax = 0.1
epsilon = 1.0



In [21]:
P, Q = differentially_private_sgd(R, d, gamma, lambd, k, emax, epsilon)

In [22]:
print('The size of P is',P.shape)
print('The size of P is',Q.shape)

The size of P is (944, 3)
The size of P is (1683, 3)


# **Recommendation Model**

user_id: The ID of the user for whom you want to generate recommendations.
R: The original ratings matrix.
P and Q: The user and item factor matrices obtained from the SGD process.
item_averages and user_effects: Arrays containing the differentially private item averages and user-specific effects, respectively.
top_n: The number of top recommendations to return.

In [32]:
num_items = R.shape[1]
print(num_items)

1683


# **Prediction Part**

In [33]:
import numpy as np

def predict_rating(user_id, item_id, P, Q, item_averages, user_effects):
    # Predict using the dot product of user and item factors from P and Q
    predicted_rating = np.dot(P[user_id], Q[item_id].T)

    adjusted_prediction = predicted_rating + item_averages[item_id] + user_effects[user_id]


    return np.clip(adjusted_prediction, 1, 5)


def recommend_similar_movies(user_id, given_item_id, R, P, Q, item_averages, user_effects, top_n=10):
    # Predict rating for the given movie and user
    given_item_predicted_rating = predict_rating(user_id, given_item_id, P, Q, item_averages, user_effects)

    # Predict ratings for all movies for the same user
    num_items = R.shape[1]
    all_predicted_ratings = np.array([
        predict_rating(user_id, item_id, P, Q, item_averages, user_effects)
        for item_id in range(num_items)
    ])

    # Calculate the absolute difference from the given movie's predicted rating
    rating_differences = np.abs(all_predicted_ratings - given_item_predicted_rating)

    # Mask out the given movie to avoid recommending it
    rating_differences[given_item_id] = np.inf

    # Mask out already rated items to avoid recommending them
    rated_items_mask = R[user_id] > 0
    rating_differences[rated_items_mask] = np.inf

    # Get the indices of the top N movies with the smallest difference
    similar_item_ids = np.argsort(rating_differences)[:top_n]

    return similar_item_ids



In [25]:
prediction = recommend_similar_movies(299,144,R, P, Q, item_averages, user_averages)
print(prediction)

[   0 1131 1130 1129 1128 1127 1126 1125 1124 1123]


In [34]:
import numpy as np

def get_actual_rating(user_id, item_id, R):
    return R[user_id, item_id]

def get_predicted_rating(user_id, item_id, P, Q, item_averages, user_effects):
    predicted_rating = np.dot(P[user_id], Q[item_id].T)
    adjusted_prediction = predicted_rating + item_averages[item_id] + user_effects[user_id]
    return np.clip(adjusted_prediction, 1, 5)

def get_similar_movies_with_ratings(user_id, given_item_id, R, P, Q, item_averages, user_effects, top_n=10):
    # Get actual rating of the given movie
    actual_rating = get_actual_rating(user_id, given_item_id, R)

    # Get private predicted rating of the given movie
    private_predicted_rating = get_predicted_rating(user_id, given_item_id, P, Q, item_averages, user_effects)

    # Get predicted ratings for all movies for the same user
    num_items = R.shape[1]
    all_predicted_ratings = np.array([
        get_predicted_rating(user_id, item_id, P, Q, item_averages, user_effects)
        for item_id in range(num_items)
    ])

    # Calculate absolute difference from the private predicted rating of the given movie
    rating_differences = np.abs(all_predicted_ratings - private_predicted_rating)

    # Sort movies based on their similarity to the private predicted rating
    similar_item_ids = np.argsort(rating_differences)[:top_n]

    # Get the actual and predicted ratings of the similar movies
    similar_movies_ratings = [get_actual_rating(user_id, item_id, R) for item_id in similar_item_ids]
    similar_movies_predicted_ratings = [all_predicted_ratings[item_id] for item_id in similar_item_ids]

    return actual_rating, private_predicted_rating, similar_item_ids, similar_movies_ratings, similar_movies_predicted_ratings

def get_majority_rating(ratings):
    # Count occurrences of each rating
    rating_counts = {}
    for rating in ratings:
        rating_counts[rating] = rating_counts.get(rating, 0) + 1

    # Find the rating with the highest count
    majority_rating = max(rating_counts, key=rating_counts.get)
    return majority_rating


user_id = 299
given_item_id = 144
actual_rating, private_predicted_rating, similar_item_ids, similar_movies_ratings, similar_movies_predicted_ratings = get_similar_movies_with_ratings(user_id, given_item_id, R, P, Q, item_averages, user_averages)
print("Actual Rating of the given movie:", actual_rating)
print("Private Predicted Rating of the given movie:", private_predicted_rating)
print("Original Ratings of 10 similar movies:")
for item_id, rating in zip(similar_item_ids, similar_movies_ratings):
    print(f"Movie ID: {item_id}, Original Rating: {rating}")

if similar_movies_ratings:
    majority_actual_rating = get_majority_rating(similar_movies_ratings)
    print("Predicted Rating for the movie:", majority_actual_rating)
else:
    print("No similar movies found.")









Actual Rating of the given movie: 4.0
Private Predicted Rating of the given movie: 1.0
Original Ratings of 10 similar movies:
Movie ID: 0, Original Rating: 0.0
Movie ID: 1128, Original Rating: 0.0
Movie ID: 1127, Original Rating: 0.0
Movie ID: 1126, Original Rating: 0.0
Movie ID: 1125, Original Rating: 0.0
Movie ID: 1124, Original Rating: 0.0
Movie ID: 1123, Original Rating: 0.0
Movie ID: 1122, Original Rating: 0.0
Movie ID: 1121, Original Rating: 0.0
Movie ID: 1120, Original Rating: 0.0
Predicted Rating for the movie: 0.0


In [35]:
import numpy as np

def get_actual_rating(user_id, item_id, R):
    return R[user_id, item_id]

def get_predicted_rating(user_id, item_id, P, Q, item_averages, user_effects):
    predicted_rating = np.dot(P[user_id], Q[item_id].T)
    adjusted_prediction = predicted_rating + item_averages[item_id] + user_effects[user_id]
    return np.clip(adjusted_prediction, 1, 5)

def get_similar_unrated_movies_ratings(user_id, given_item_id, R, P, Q, item_averages, user_effects, top_n=10):
    # Get actual rating of the given movie
    actual_rating = get_actual_rating(user_id, given_item_id, R)

    # Get predicted rating of the given movie
    predicted_rating = get_predicted_rating(user_id, given_item_id, P, Q, item_averages, user_effects)

    # Predict ratings for all unrated movies for the same user
    unrated_item_ids = np.where(R[user_id] == 0)[0]
    all_predicted_ratings = np.array([
        get_predicted_rating(user_id, item_id, P, Q, item_averages, user_effects)
        for item_id in unrated_item_ids
    ])

    # Calculate absolute difference from the predicted rating of the given movie
    rating_differences = np.abs(all_predicted_ratings - predicted_rating)

    # Sort unrated movies based on their similarity to the given movie's predicted rating
    similar_item_ids = unrated_item_ids[np.argsort(rating_differences)[:top_n]]

    # Get the predicted ratings of the similar unrated movies
    similar_movies_predicted_ratings = [get_predicted_rating(user_id, item_id, P, Q, item_averages, user_effects) for item_id in similar_item_ids]

    return actual_rating, predicted_rating, similar_item_ids, similar_movies_predicted_ratings

user_id = 299
given_item_id = 144
actual_rating, predicted_rating, similar_item_ids, similar_movies_predicted_ratings = get_similar_unrated_movies_ratings(user_id, given_item_id, R, P, Q, item_averages, user_averages)
print("Actual Rating of the given movie:", actual_rating)
print("Predicted Rating of the given movie:", predicted_rating)
print("Predicted Ratings of 10 similar unrated movies:")
for item_id, rating in zip(similar_item_ids, similar_movies_predicted_ratings):
    print(f"Movie ID: {item_id}, Predicted Rating: {rating}")


Actual Rating of the given movie: 4.0
Predicted Rating of the given movie: 1.0
Predicted Ratings of 10 similar unrated movies:
Movie ID: 0, Predicted Rating: 1.0
Movie ID: 1209, Predicted Rating: 1.0
Movie ID: 1208, Predicted Rating: 1.0
Movie ID: 1207, Predicted Rating: 1.0
Movie ID: 1206, Predicted Rating: 1.0
Movie ID: 1205, Predicted Rating: 1.0
Movie ID: 1204, Predicted Rating: 1.0
Movie ID: 1203, Predicted Rating: 1.0
Movie ID: 1202, Predicted Rating: 1.0
Movie ID: 1201, Predicted Rating: 1.0
